In [2]:
from PIL import Image
import os
from collections import defaultdict
from pathlib import Path

In [4]:
def categorize_images_by_size(input_folder):
    # Define size categories (width x height)
    categories = [
        ("< 24x24", 0, 24),
        ("24x24 - 32x32", 24, 32),
        ("32x32 - 64x64", 32, 64),
        ("64x64 - 128x128", 64, 128),
        ("128x128 - 256x256", 128, 256),
        ("256x256 - 512x512", 256, 512),
        ("> 512x512", 512, float('inf'))
    ]
    
    # Track counts and store image details
    category_counts = defaultdict(int)
    category_images = defaultdict(list)
    
    # Supported image formats
    valid_extensions = {'.png', '.jpg', '.jpeg'}
    
    total_images = 0
    
    print("Analyzing images...\n")
    
    # Process each image
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        if not os.path.isfile(file_path):
            continue
            
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_extensions:
            continue
        
        try:
            img = Image.open(file_path)
            width, height = img.size
            max_dimension = max(width, height)
            
            # Find appropriate category
            for cat_name, min_size, max_size in categories:
                if min_size <= max_dimension < max_size:
                    category_counts[cat_name] += 1
                    category_images[cat_name].append(f"{filename} ({width}x{height})")
                    break
            
            total_images += 1
            
        except Exception as e:
            print(f"✗ Error reading {filename}: {str(e)}")
    
    # Print frequency table
    print("=" * 70)
    print("IMAGE SIZE DISTRIBUTION")
    print("=" * 70)
    print(f"{'Category':<20} {'Count':>10} {'Percentage':>12} {'Bar':>20}")
    print("-" * 70)
    
    for cat_name, min_size, max_size in categories:
        count = category_counts[cat_name]
        percentage = (count / total_images * 100) if total_images > 0 else 0
        
        # Create visual bar
        bar_length = int(percentage / 5)  # Scale: 5% = 1 char
        bar = "█" * bar_length
        
        print(f"{cat_name:<20} {count:>10} {percentage:>11.1f}% {bar:>20}")
    
    print("-" * 70)
    print(f"{'TOTAL':<20} {total_images:>10} {100.0:>11.1f}%")
    print("=" * 70)
    
    # Print detailed breakdown (optional)
    print("\n" + "=" * 70)
    print("DETAILED BREAKDOWN BY CATEGORY")
    print("=" * 70)
    
    for cat_name, min_size, max_size in categories:
        if category_counts[cat_name] > 0:
            print(f"\n{cat_name}: {category_counts[cat_name]} images")
            print("-" * 70)
            for img_info in category_images[cat_name][:10]:  # Show first 10
                print(f"  • {img_info}")
            
            if len(category_images[cat_name]) > 10:
                remaining = len(category_images[cat_name]) - 10
                print(f"  ... and {remaining} more")
    
    print("\n" + "=" * 70)
    
    return category_counts, total_images


# Usage example
input_folder = "./src"

if not os.path.exists(input_folder):
    print(f"Error: Folder '{input_folder}' not found!")
    print("Please update the input_folder path.")
else:
    categorize_images_by_size(input_folder)

Analyzing images...

IMAGE SIZE DISTRIBUTION
Category                  Count   Percentage                  Bar
----------------------------------------------------------------------
< 24x24                      91         2.0%                     
24x24 - 32x32                21         0.5%                     
32x32 - 64x64               472        10.1%                   ██
64x64 - 128x128            1240        26.6%                █████
128x128 - 256x256          1610        34.5%               ██████
256x256 - 512x512           884        19.0%                  ███
> 512x512                   344         7.4%                    █
----------------------------------------------------------------------
TOTAL                      4662       100.0%

DETAILED BREAKDOWN BY CATEGORY

< 24x24: 91 images
----------------------------------------------------------------------
  • 002-lospec-dailies-286456.png (16x16)
  • fresno-nightcrawler-082838.png (16x15)
  • green-slime-040105.png (16x1

In [6]:
def categorize_images_by_aspect_ratio(input_folder):
    # Define aspect ratio categories
    # Format: (name, min_ratio, max_ratio)
    categories = [
        ("Square (1:1)", 0.95, 1.05),           # Nearly square
        ("Slightly Wide (5:4 to 4:3)", 1.05, 1.4),
        ("Wide (16:10 to 16:9)", 1.4, 1.9),
        ("Very Wide (2:1+)", 1.9, float('inf')),
        ("Slightly Tall (4:5 to 3:4)", 0.71, 0.95),
        ("Tall (9:16 to 10:16)", 0.53, 0.71),
        ("Very Tall (1:2+)", 0, 0.53)
    ]
    
    # Track counts and store image details
    category_counts = defaultdict(int)
    category_images = defaultdict(list)
    
    # Supported image formats
    valid_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.gif'}
    
    total_images = 0
    
    print("Analyzing aspect ratios...\n")
    
    # Process each image
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        if not os.path.isfile(file_path):
            continue
            
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_extensions:
            continue
        
        try:
            img = Image.open(file_path)
            width, height = img.size
            
            # Calculate aspect ratio (width / height)
            aspect_ratio = width / height if height > 0 else 0
            
            # Find appropriate category
            for cat_name, min_ratio, max_ratio in categories:
                if min_ratio <= aspect_ratio < max_ratio:
                    category_counts[cat_name] += 1
                    category_images[cat_name].append(
                        f"{filename} ({width}x{height}, ratio: {aspect_ratio:.2f})"
                    )
                    break
            
            total_images += 1
            
        except Exception as e:
            print(f"✗ Error reading {filename}: {str(e)}")
    
    # Print frequency table
    print("=" * 75)
    print("IMAGE ASPECT RATIO DISTRIBUTION")
    print("=" * 75)
    print(f"{'Category':<30} {'Count':>10} {'Percentage':>12} {'Bar':>15}")
    print("-" * 75)
    
    for cat_name, min_ratio, max_ratio in categories:
        count = category_counts[cat_name]
        percentage = (count / total_images * 100) if total_images > 0 else 0
        
        # Create visual bar
        bar_length = int(percentage / 5)  # Scale: 5% = 1 char
        bar = "█" * bar_length
        
        print(f"{cat_name:<30} {count:>10} {percentage:>11.1f}% {bar:>15}")
    
    print("-" * 75)
    print(f"{'TOTAL':<30} {total_images:>10} {100.0:>11.1f}%")
    print("=" * 75)
    
    # Print detailed breakdown
    print("\n" + "=" * 75)
    print("DETAILED BREAKDOWN BY CATEGORY")
    print("=" * 75)
    
    for cat_name, min_ratio, max_ratio in categories:
        if category_counts[cat_name] > 0:
            print(f"\n{cat_name}: {category_counts[cat_name]} images")
            print("-" * 75)
            for img_info in category_images[cat_name][:10]:  # Show first 10
                print(f"  • {img_info}")
            
            if len(category_images[cat_name]) > 10:
                remaining = len(category_images[cat_name]) - 10
                print(f"  ... and {remaining} more")
    
    print("\n" + "=" * 75)
    
    # Summary warnings
    print("\n⚠️  ASPECT RATIO WARNINGS:")
    print("-" * 75)
    
    extreme_wide = category_counts["Very Wide (2:1+)"]
    extreme_tall = category_counts["Very Tall (1:2+)"]
    
    if extreme_wide > 0:
        print(f"  • {extreme_wide} images are very wide (ratio > 1.9)")
        print(f"    These may look stretched when converted to square.")
    
    if extreme_tall > 0:
        print(f"  • {extreme_tall} images are very tall (ratio < 0.53)")
        print(f"    These may look stretched when converted to square.")
    
    if extreme_wide == 0 and extreme_tall == 0:
        print(f"  ✓ No extreme aspect ratios detected!")
    
    square_like = category_counts["Square (1:1)"]
    print(f"\n  ✓ {square_like} images are already square or nearly square")
    print("=" * 75)
    
    return category_counts, total_images


# Usage example
input_folder = "./src"

if not os.path.exists(input_folder):
    print(f"Error: Folder '{input_folder}' not found!")
    print("Please update the input_folder path.")
else:
    categorize_images_by_aspect_ratio(input_folder)

Analyzing aspect ratios...

IMAGE ASPECT RATIO DISTRIBUTION
Category                            Count   Percentage             Bar
---------------------------------------------------------------------------
Square (1:1)                         3014        54.8%      ██████████
Slightly Wide (5:4 to 4:3)            583        10.6%              ██
Wide (16:10 to 16:9)                  682        12.4%              ██
Very Wide (2:1+)                      291         5.3%               █
Slightly Tall (4:5 to 3:4)            500         9.1%               █
Tall (9:16 to 10:16)                  332         6.0%               █
Very Tall (1:2+)                       94         1.7%                
---------------------------------------------------------------------------
TOTAL                                5496       100.0%

DETAILED BREAKDOWN BY CATEGORY

Square (1:1): 3014 images
---------------------------------------------------------------------------
  • his-not-so-smart-as-hi-thi

In [5]:
def process_pixel_art(input_folder, output_folder, target_size=128):
    # Create output folder if it doesn't exist
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    # Supported image formats
    valid_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.gif'}
    
    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        # Skip if not a file or not a valid image
        if not os.path.isfile(file_path):
            continue
            
        ext = os.path.splitext(filename)[1].lower()
        if ext not in valid_extensions:
            continue
        
        try:
            # Open image
            img = Image.open(file_path)
            
            # Handle transparency: replace with black if PNG has alpha channel
            if ext == '.png' and (img.mode == 'RGBA' or img.mode == 'LA' or 'transparency' in img.info):
                # Create black background
                background = Image.new('RGB', img.size, (0, 0, 0))
                # Convert image to RGBA if needed
                if img.mode != 'RGBA':
                    img = img.convert('RGBA')
                # Composite image over black background
                background.paste(img, mask=img.split()[3])  # Use alpha channel as mask
                img = background
            else:
                # Convert to RGB for consistent color fidelity
                if img.mode != 'RGB':
                    img = img.convert('RGB')
            
            width, height = img.size
            
            # Skip images larger than 512x512
            if width > 512 or height > 512:
                print(f"⊗ Skipped: {filename} ({width}x{height} - exceeds 512x512 limit)")
                continue
            
            # Calculate aspect ratio and skip extreme ratios
            aspect_ratio = width / height if height > 0 else 0
            
            # Skip very wide images (ratio >= 1.9)
            if aspect_ratio >= 1.9:
                print(f"⊗ Skipped: {filename} ({width}x{height}, ratio: {aspect_ratio:.2f} - too wide)")
                continue
            
            # Skip very tall images (ratio <= 0.53)
            if aspect_ratio <= 0.53:
                print(f"⊗ Skipped: {filename} ({width}x{height}, ratio: {aspect_ratio:.2f} - too tall)")
                continue
            
            # Case 1: Image needs to be cropped (larger than target)
            if width > target_size or height > target_size:
                # Step 1: Scale down so the smaller dimension equals target_size
                # This maintains aspect ratio without distortion
                if width < height:
                    # Width is smaller, scale so width = target_size
                    scale_factor = target_size / width
                    new_width = target_size
                    new_height = int(height * scale_factor)
                else:
                    # Height is smaller (or equal), scale so height = target_size
                    scale_factor = target_size / height
                    new_height = target_size
                    new_width = int(width * scale_factor)
                
                # Resize with NEAREST to maintain sharp pixel art edges
                img = img.resize((new_width, new_height), Image.NEAREST)
                
                # Step 2: Crop the larger dimension from center if still > target_size
                width, height = img.size
                
                if width > target_size or height > target_size:
                    left = (width - target_size) // 2
                    top = (height - target_size) // 2
                    right = left + target_size
                    bottom = top + target_size
                    
                    img = img.crop((left, top, right, bottom))
            
            # Case 2: Image needs to be scaled up (smaller than target)
            elif width < target_size or height < target_size:
                # Use NEAREST neighbor to maintain sharp pixel art edges
                img = img.resize((target_size, target_size), Image.NEAREST)
            
            # Case 3: Image is already the target size
            # (no action needed)
            
            # Save processed image
            output_path = os.path.join(output_folder, filename)
            
            # Save as PNG to preserve color fidelity (no compression artifacts)
            if ext != '.png':
                output_path = os.path.splitext(output_path)[0] + '.png'
            
            img.save(output_path, 'PNG', optimize=False)
            print(f"✓ Processed: {filename} ({width}x{height} → {target_size}x{target_size})")
            
        except Exception as e:
            print(f"✗ Error processing {filename}: {str(e)}")

input_folder = "./src"
output_folder = "./final"

process_pixel_art(input_folder, output_folder, target_size=64)
print("\nProcessing complete!")

✓ Processed: his-not-so-smart-as-hi-think-he-is-396583.png (32x32 → 64x64)
⊗ Skipped: flipside-boss-fight-animation-898573.gif (1100x1320 - exceeds 512x512 limit)
✓ Processed: oh-no-276053.png (64x64 → 64x64)
✓ Processed: golem-of-hill-073378.png (29x29 → 64x64)
✓ Processed: windows-xp-for-c64-074511.png (102x64 → 64x64)
✓ Processed: what-393777.png (102x64 → 64x64)
✓ Processed: a-game-du-you-wanna-584037.png (64x64 → 64x64)
✓ Processed: starry-night-434230.png (93x64 → 64x64)
✓ Processed: last-ronin-pixelart-fanart-227054.png (96x64 → 64x64)
✓ Processed: coming-soon-017725.png (64x80 → 64x64)
✓ Processed: the-color-out-of-space-242366.png (64x64 → 64x64)
⊗ Skipped: first-day-of-school-466173.png (166x333, ratio: 0.50 - too tall)
✓ Processed: planet-995488.png (32x32 → 64x64)
✓ Processed: mountains-630341.png (64x64 → 64x64)
✓ Processed: bubble-moon-111779.png (64x64 → 64x64)
✓ Processed: dragon80-731973.gif (64x64 → 64x64)
✓ Processed: deidara-988234.png (64x64 → 64x64)
✓ Processed: w